In [1]:
import sys
from pathlib import Path

# Add the project src folder to Python's path
root = Path().resolve().parent
print(root)
sys.path.append(str(root))

C:\Users\Ariel\Projects\email-security-copilot


In [2]:
import os, pandas as pd
from src.email_security_copilot.config import ProjectConfig
from src.email_security_copilot.data.data import EmailDataset
from src.email_security_copilot.data.textPreprocessor import TextPreprocessor
# from src.utils import combine_subject_content
from src.email_security_copilot.data.dataset import SpamFrameDataset

c:\Users\Ariel\Projects\email-security-copilot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cfg = ProjectConfig()
cfg.validate()
print("Config OK")

Config OK


In [5]:
spam_dir = cfg.data.spam_dir
ham_dir  = cfg.data.ham_dir

assert os.path.isdir(spam_dir), f"Missing dir: {spam_dir}"
assert os.path.isdir(ham_dir),  f"Missing dir: {ham_dir}"

spam_df = EmailDataset(spam_dir, label="spam").load()
ham_df  = EmailDataset(ham_dir,  label="ham").load()
df = pd.concat([spam_df, ham_df], ignore_index=True)

print(
    f"Loaded: {len(df)} emails "
    f"({(df.label=='spam').sum()} spam, {(df.label=='ham').sum()} ham)"
)

Loaded: 3948 emails (1397 spam, 2551 ham)


In [19]:
from src.email_security_copilot.features.features import EmailFeatureClass
features_df = EmailFeatureClass()(df)

In [26]:
cols = ['sender', 'recipient', 'num_cc', 'subject', 'content', 
        'subject_len', 'content_len', 'has_link', 'num_hops', 'label']
raw_df = features_df[cols]

In [ ]:
# df["text_raw"] = df.apply(combine_subject_content, axis=1)

# pre = TextPreprocessor(cfg)
# df["text_raw_clean"] = df["text_raw"].fillna("").map(pre)

# assert df["text_raw_clean"].isna().sum() == 0, "NaNs in cleaned content"
# non_empty_rate = (df["text_raw_clean"].str.len() > 0).mean()
# print(f"Cleaned text — non-empty rate: {non_empty_rate:.2%}")

# df_clean = df[["text_raw_clean", "label"]].copy()

Loaded: 3948 emails (1397 spam, 2551 ham)
Cleaned text — non-empty rate: 100.00%


In [ ]:
# llm_dataset = SpamFrameDataset(
#     df=df_clean,
#     cfg=cfg,
# )
# len(llm_dataset), llm_dataset[0].keys()

(3948, dict_keys(['input_ids', 'attention_mask', 'labels']))

In [ ]:
# from src.utils import combine_subject_content
# df["text_raw"] = df.apply(combine_subject_content, axis=1)

# pre = TextPreprocessor(cfg)
# df["text_raw_clean"] = df["text_raw"].fillna("").map(pre)

# assert df["text_raw_clean"].isna().sum() == 0, "NaNs in cleaned content"
# non_empty_rate = (df["text_raw_clean"].str.len() > 0).mean()
# print(f"Cleaned text — non-empty rate: {non_empty_rate:.2%}")

Cleaned text — non-empty rate: 100.00%
